<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/07_Avg_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import *
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE, RFECV
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import warnings
import re
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('car_prices_clean.csv')
df.head(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,brand_rank,model_rank,fuel_type_rank,engine_rank,transmission_rank,ext_col_rank,int_col_rank,accident_rank,motor_rank,fuel_rank
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,...,31,1256,4,821,28,91,14,2,1,3


In [3]:
columns = ['model_year',
           'milage',
           'hp',
           'litr',
           'Cylinder',
           'brand_rank',
           'model_rank',
           'fuel_type_rank',
           'engine_rank',
           'transmission_rank',
           'ext_col_rank',
           'int_col_rank',
           'accident_rank',
           'motor_rank',
           'fuel_rank'
           ]

In [4]:
X = df[columns]
y = df['price']
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = pd.DataFrame(poly.fit_transform(X), columns=poly.get_feature_names_out(X.columns))

In [5]:
cv = KFold(n_splits=6, shuffle=True, random_state=42)

linear_model = LinearRegression()

rfecv = RFECV(estimator=linear_model, step=1, cv=cv, scoring='neg_mean_squared_error', verbose=2)

rfecv.fit(X_poly, y)

Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
Fitting estimator with 127 features.
Fitting estimator with 126 features.
Fitting estimator with 125 features.
Fitting estimator with 124 features.
Fitting estimator with 123 features.
Fitting estimator with 122 features.
Fitting estimator with 121 features.
Fitting estimator with 120 features.
Fitting estimator with 119 features.
Fitting estimator with 118 features.
Fitting estimator with 117 features.
Fitting estimator with 116 features.
Fitting estimator with 115 features.
Fitting estimator with 114 features.
Fitting estimator with 113 features.
Fitting estimator with 112 features.
Fitting estimator with 111 features.
Fitting estimator with 110 features.
Fitting estimator with 109 features.
F

RFECV(cv=KFold(n_splits=6, random_state=42, shuffle=True),
      estimator=LinearRegression(), scoring='neg_mean_squared_error',
      verbose=2)

In [6]:
optimal_num_features = rfecv.n_features_
X_select = X_poly[X_poly.columns[rfecv.support_]]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



estimators = [
    ('lr', Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LinearRegression())
    ])),
    ('ridge', Pipeline([
        ('scaler', StandardScaler()),
        ('ridge', Ridge())
    ])),
    ('lasso', Pipeline([
        ('scaler', StandardScaler()),
        ('lasso', Lasso())
    ])),
    ('elasticnet', Pipeline([
        ('scaler', StandardScaler()),
        ('elasticnet', ElasticNet())
    ]))
]

param_grid = {
    'final_estimator__alpha': [0.1, 0.5, 1, 5, 10, 100],  # Ridge uchun
    'ridge__ridge__alpha': [0.1, 1, 10],  # Ridge modelining alpha parametri
    'lasso__lasso__alpha': [0.01, 0.1, 1],  # Lasso modelining alpha parametri
    'elasticnet__elasticnet__alpha': [0.01, 0.1, 1],  # ElasticNet modelining alpha parametri
    'elasticnet__elasticnet__l1_ratio': [0.1, 0.5, 0.9]  # ElasticNet modelining l1_ratio parametri
}

stacking_model = StackingRegressor(estimators=estimators, final_estimator=Ridge())

random_search = RandomizedSearchCV(stacking_model,
                                   param_distributions=param_grid,
                                  #  n_iter=10,
                                  #  cv=5,
                                  #  random_state=0
                                   )


random_search.fit(X_train, y_train)


random_search.best_params_

{'ridge__ridge__alpha': 10,
 'lasso__lasso__alpha': 0.01,
 'final_estimator__alpha': 100,
 'elasticnet__elasticnet__l1_ratio': 0.5,
 'elasticnet__elasticnet__alpha': 0.01}